<a href="https://colab.research.google.com/github/dtht2d/deep-learning/blob/main/codes/neural_network_fit/bodyfat__fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%autosave 30

Autosaving every 30 seconds


# **Objective**
---
- Create similarity **nftool** (MATLAB's Neural Network Toolbox) function in Python
- We have data from a health clinic and our goal is to design a neural network that can predict the percentage of body fat of a person, based on 13 anatomical measurements. The dataset contains information from 252 individuals, with measurements for each of the 13 variables, as well as their corresponding percentage of body fat. The objective is to train a neural network to predict body fat percentage based on the input variables.
- Observe the output by changing the training/validation/testing percentages and the number of neurons. 

## **1. Background**


### **Neural Network Fitting Algorithm**
A two-layer feed-forward network trained with Levenberg-Marquardt is a neural network architecture commonly used for solving regression problems. 

**Two-layer feed-forward neural network architecture:**
  - The information flows in a single direction from the input layer to the output layer.
  - There is one input layer, one output layer, and at least one hidden layer in between. The hidden layer(s) contain nodes that apply transformations to the input data, which are then fed forward to the output layer.

- **Levenberg-Marquardt algorithm:**
  - A popular optimization algorithm used to train neural networks by optimizing the weights and biases of the network.
  - It is a combination of two other optimization algorithms: the Gauss-Newton algorithm and the Gradient Descent algorithm
  - Particularly effective for training feed-forward neural networks with one or more hidden layers.
  - For mathemtical equations reference: http://web.eecs.utk.edu/~mjr/ECE505/Miscellaneous/levmar.pdf


## **2. Methods**


### Install packages


In [10]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import libraries

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
path = "/content/bodyfat.csv"

In [14]:
df = pd.read_csv(path)
print (df)

     Density  BodyFat  Age  Weight  Height  Neck  Chest  Abdomen    Hip  \
0     1.0708     12.3   23  154.25   67.75  36.2   93.1     85.2   94.5   
1     1.0853      6.1   22  173.25   72.25  38.5   93.6     83.0   98.7   
2     1.0414     25.3   22  154.00   66.25  34.0   95.8     87.9   99.2   
3     1.0751     10.4   26  184.75   72.25  37.4  101.8     86.4  101.2   
4     1.0340     28.7   24  184.25   71.25  34.4   97.3    100.0  101.9   
..       ...      ...  ...     ...     ...   ...    ...      ...    ...   
247   1.0736     11.0   70  134.25   67.00  34.9   89.2     83.6   88.8   
248   1.0236     33.6   72  201.00   69.75  40.9  108.5    105.0  104.5   
249   1.0328     29.3   72  186.75   66.00  38.9  111.1    111.5  101.7   
250   1.0399     26.0   72  190.75   70.50  38.9  108.3    101.3   97.8   
251   1.0271     31.9   74  207.50   70.00  40.8  112.4    108.5  107.1   

     Thigh  Knee  Ankle  Biceps  Forearm  Wrist  
0     59.0  37.3   21.9    32.0     27.4   17.1  

BodyFat" is located in the third column of the dataset, with an index of [2]. To prepare the data for training a neural network to predict body fat percentage, we need to select the input features (denoted as "X") that will be used to make the predictions. To do this, we should choose all the features in the dataset except for the "BodyFat" column. 

In [19]:
X = df.iloc[:, 2:].to_numpy()
y = df['BodyFat'].to_numpy()
print(X)
print(X.shape)
print(y)
print(y.shape)

[[ 23.   154.25  67.75 ...  32.    27.4   17.1 ]
 [ 22.   173.25  72.25 ...  30.5   28.9   18.2 ]
 [ 22.   154.    66.25 ...  28.8   25.2   16.6 ]
 ...
 [ 72.   186.75  66.   ...  31.3   27.2   18.  ]
 [ 72.   190.75  70.5  ...  30.5   29.4   19.8 ]
 [ 74.   207.5   70.   ...  33.7   30.    20.9 ]]
(252, 13)
[12.3  6.1 25.3 10.4 28.7 20.9 19.2 12.4  4.1 11.7  7.1  7.8 20.8 21.2
 22.1 20.9 29.  22.9 16.  16.5 19.1 15.2 15.6 17.7 14.   3.7  7.9 22.9
  3.7  8.8 11.9  5.7 11.8 21.3 32.3 40.1 24.2 28.4 35.2 32.6 34.5 32.9
 31.6 32.   7.7 13.9 10.8  5.6 13.6  4.  10.2  6.6  8.   6.3  3.9 22.6
 20.4 28.  31.5 24.6 26.1 29.8 30.7 25.8 32.3 30.  21.5 13.8  6.3 12.9
 24.3  8.8  8.5 13.5 11.8 18.5  8.8 22.2 21.5 18.8 31.4 26.8 18.4 27.
 27.  26.6 14.9 23.1  8.3 14.1 20.5 18.2  8.5 24.9  9.  17.4  9.6 11.3
 17.8 22.2 21.2 20.4 20.1 22.3 25.4 18.  19.3 18.3 17.3 21.4 19.7 28.
 22.1 21.3 26.7 16.7 20.1 13.9 25.8 18.1 27.9 25.3 14.7 16.  13.8 17.5
 27.2 17.4 20.8 14.9 18.1 22.7 23.6 26.1 24.4 27.1 21